In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df_amazon_sr = pd.read_csv(
    '../data/raw_csv_files/Amazon Sale Report.csv', low_memory=False)

In [ ]:
df_amazon_sr.head()

In [ ]:
# Format columns
df_amazon_sr.columns = (
    df_amazon_sr.columns
    .str.strip()                                # Remove leading/trailing whitespace
    .str.lower()                                # Convert to lowercase
    .str.replace(r'[^\w\s]', '', regex=True)    # Remove special characters like - or .
    .str.replace(r'\s+', '_', regex=True)       # Replace spaces with underscores
)

In [ ]:
df_amazon_sr.nunique()

In [ ]:
# Drop unnecessary columns
cols_to_drop = [
    'index',
    'asin',
    'shipcountry',
    'unnamed_22'
]
df_amazon_sr.drop(columns=cols_to_drop, inplace=True)

In [ ]:
df_amazon_sr['sales_channel'].value_counts()

In [ ]:
# Drop 124 rows from non-Amazon sales_channel (all unshipped, not relevant)
df_amazon_sr = df_amazon_sr[df_amazon_sr['sales_channel'] == 'Amazon.in']

In [ ]:
# Drop 'sales_channel' now that it's uniform

# Filter and make a safe copy
df_amazon_sr = df_amazon_sr[df_amazon_sr['sales_channel'] == 'Amazon.in'].copy()

# Now it's safe to drop the column
df_amazon_sr.drop(columns=['sales_channel'], inplace=True)

In [ ]:
df_amazon_sr.info()

In [ ]:
# Convert some data types

In [ ]:
# Convert date to datetime64
df_amazon_sr['date'] = pd.to_datetime(
    df_amazon_sr['date'],
    format='%m-%d-%y',     # Specify expected format
    errors='coerce'        # Handle bad entries safely
)
print(df_amazon_sr['date'].dtypes)

In [ ]:
# Convert postal codes to strings so we don't lose any formatting (like leading zeros)
df_amazon_sr['shippostalcode'] = (
    df_amazon_sr['shippostalcode']
    .astype('Int64')   # keep nulls intact
    .astype(str)
)

In [ ]:
# Confirm
print(df_amazon_sr[['date', 'shippostalcode']].dtypes)

In [ ]:
# Show date rage of data
print('Date Range:')
print('Min:', df_amazon_sr['date'].min())
print('Max:', df_amazon_sr['date'].max())

In [ ]:
# In India, Q1 FY23 = April 1, 2022 to June 30, 2022
# The original data starts on March 31 and ends on June 29 — likely due to time zone differences or US-based reporting
# Shift all dates forward by one day to align cleanly with Indian fiscal Q1 FY23

In [ ]:
df_amazon_sr['date'] = df_amazon_sr['date'] + pd.Timedelta(days=1)

In [ ]:
# Show date rage of data
print('Date Range:')
print('Min:', df_amazon_sr['date'].min())
print('Max:', df_amazon_sr['date'].max())

In [ ]:
# Order the dataframe by date
df_amazon_sr = df_amazon_sr.sort_values(by='date').reset_index(drop=True)

In [ ]:
df_amazon_sr.describe()

In [ ]:
# Rename some columns for claity
df_amazon_sr.rename(columns={
    'shipcity': 'city',
    'shipstate': 'state',
    'shippostalcode': 'postal_code',
    'shipservicelevel': 'shipping_method',
    'promotionids': 'promo_id',
    'fulfilledby': 'fulfilled_by'
}, inplace=True)

In [ ]:
# Format Title Case for category values (consistency) & remove any white spaces
df_amazon_sr['category'] = df_amazon_sr['category'].str.strip().str.title()

In [ ]:
# Re-eheck unique promo_ids
df_amazon_sr['promo_id'].nunique()

In [ ]:
# Looks very messy
df_amazon_sr['promo_id'].dropna().unique()[:10]

In [ ]:
# Cut out everything after 'Free-Financing' (it's meaningless) and assign to new column
df_amazon_sr['promo_group'] = df_amazon_sr['promo_id'].str.extract(r'^(.*?Free-Financing)')

In [ ]:
# Fill in all NaNs in promo_group with 'None'
df_amazon_sr['promo_group'] = df_amazon_sr['promo_group'].fillna('None')

In [ ]:
# Check value counts
df_amazon_sr['promo_group'].value_counts()

In [ ]:
# Check number of uniques 
df_amazon_sr['promo_group'].nunique()
# 5787 -> 2

In [ ]:
# Drop the original, messy 'promo_id' column
df_amazon_sr.drop(columns=['promo_id'], inplace=True)

In [ ]:
# Get original column name back; rename 'promo_group' to 'promo_id'
df_amazon_sr.rename(columns={'promo_group': 'promo_id'}, inplace=True)

In [ ]:
# Fill in all NaNs in fulfilled_by with 'Other'
df_amazon_sr['fulfilled_by'] = df_amazon_sr['fulfilled_by'].fillna('Other')

In [ ]:
# Clean up 'city' & 'state' columns now

In [ ]:
# First, format city & state columns to have same Title Case, strip away white space
df_amazon_sr['city'] = df_amazon_sr['city'].str.strip().str.title()
df_amazon_sr['state'] = df_amazon_sr['state'].str.strip().str.title()

In [ ]:
# Start with 'state'
df_amazon_sr['state'].nunique()

In [ ]:
# View list of unique values
sorted(df_amazon_sr['state'].dropna().unique().tolist())

In [ ]:
# Standardize common spelling errors and abbreviations in 'state'
state_corrections = {
    'Rajshthan': 'Rajasthan',
    'Rajsthan': 'Rajasthan',
    'Rj': 'Rajasthan',
    'Orissa': 'Odisha',                                             # Official state name
    'Pondicherry': 'Puducherry',
    'Pb': 'Punjab',
    'Punjab/Mohali/Zirakpur': 'Punjab',                             # Keep only state
    'New Delhi': 'Delhi',                                           # Normalize NCR variant
    'Nl': 'Nagaland',
    'Ar': 'Arunachal Pradesh',
    'Apo': None,                                                    # Likely invalid
    'Dadra And Nagar': 'Dadra And Nagar Haveli And Daman And Diu'   # Official UT name
}
df_amazon_sr['state'] = df_amazon_sr['state'].replace(state_corrections)

In [ ]:
df_amazon_sr['state'].nunique()

In [ ]:
df_amazon_sr['state'].isnull().sum()

In [ ]:
df_amazon_sr[df_amazon_sr['state'].isnull()]

In [ ]:
# Dropped 34 rows missing both 'city' and 'state' — can't be used for location analysis
df_amazon_sr = df_amazon_sr[~(df_amazon_sr['state'].isnull() & df_amazon_sr['city'].isnull())]

In [ ]:
# Check
sorted(df_amazon_sr['state'].dropna().unique().tolist())

In [ ]:
# Final Check
sorted(df_amazon_sr['state'].dropna().unique().tolist())

In [ ]:
# Now 'city'
df_amazon_sr['city'].nunique()

In [ ]:
df_amazon_sr['postal_code'].nunique()

In [ ]:
lookup_postal = (
    df_amazon_sr[['postal_code', 'state', 'city']]
    .drop_duplicates()
    .reset_index(drop=True)
)

In [ ]:
lookup_postal.shape

In [ ]:
lookup_postal.head()

In [ ]:
# Read the file (GeoNames, tab-separated, no header)
geo_postal = pd.read_csv(
    '../data/geonames_india.txt',   # ← update this path
    sep='\t',
    header=None,
    dtype={1: str}  # ensure postal_code keeps leading zeros
)

In [ ]:
geo_postal.head()

In [ ]:
# Assign column names (from the GeoNames readme.txt file)
geo_postal.columns = [
    'country_code', 'postal_code', 'place_name', 'admin_name1', 'admin_code1',
    'admin_name2', 'admin_code2', 'admin_name3', 'admin_code3',
    'latitude', 'longitude', 'accuracy'
]

In [ ]:
geo_postal.head()

In [ ]:
geo_postal.info()

In [ ]:
# Filter only rows with valid lat/lon and place_name
geo_filtered = geo_postal[
    geo_postal['place_name'].notnull() &
    geo_postal['latitude'].notnull() &
    geo_postal['longitude'].notnull()
].copy()

# Keep best (lowest) accuracy row for each (postal_code, admin_name1)
geo_grouped = (
    geo_filtered.sort_values(by='accuracy')
    .groupby(['postal_code', 'admin_name1'], as_index=False)
    .first()
)
# Rename columns for merge compatibility
geo_grouped.rename(columns={
    'place_name': 'city_geo',
    'admin_name1': 'state'
}, inplace=True)

In [ ]:
# Let's see if GeoNames format of the state values is = to ours
set(geo_grouped['state'].unique()) == set(df_amazon_sr['state'].unique())

In [ ]:
set(df_amazon_sr['state'].unique()) - set(geo_grouped['state'].unique())


In [ ]:
# Match Amazon state names to GeoNames format
amazon_state_corrections = {
    'Andaman & Nicobar': 'Andaman & Nicobar Islands',
    'Chhattisgarh': 'Chattisgarh',
    'Dadra And Nagar Haveli And Daman And Diu': 'Dadra and Nagar Haveli and Daman and Diu',
    'Ladakh': 'Jammu & Kashmir',
    'Puducherry': 'Pondicherry'
}
df_amazon_sr['state'] = df_amazon_sr['state'].replace(amazon_state_corrections)

In [ ]:
# Merge enriched city and coordinates into main DataFrame
df_amazon_sr = df_amazon_sr.merge(
    geo_grouped[['postal_code', 'state', 'city_geo', 'latitude', 'longitude']],
    on=['postal_code', 'state'],
    how='left'
)

In [ ]:
# Let's see if GeoNames format of the state values is now = to ours
set(geo_grouped['state'].unique()) == set(df_amazon_sr['state'].unique())

In [ ]:
# It checks out, only value is 'None' which geo_grouped does not have
set(df_amazon_sr['state'].unique()) - set(geo_grouped['state'].unique())

In [ ]:
# Compare cleaned 'city' to enriched 'city_geo'
df_amazon_sr['city_match'] = (
    df_amazon_sr['city'] == df_amazon_sr['city_geo']
)

# Check match counts
df_amazon_sr['city_match'].value_counts()


In [ ]:
df_amazon_sr.head()

In [ ]:
# Use city_geo if available, fallback to original city if not
df_amazon_sr['city_final'] = df_amazon_sr['city_geo'].fillna(df_amazon_sr['city'])

In [ ]:
df_amazon_sr.drop(columns=['city_geo', 'city_match'], inplace=True)

In [ ]:
# Quick check on the structure
df_amazon_sr[['city_final', 'state', 'postal_code', 'latitude', 'longitude']].head()

In [ ]:
# Drop old city column and replace with trusted GeoNames-based city
df_amazon_sr.drop(columns=['city'], inplace=True)
df_amazon_sr.rename(columns={'city_final': 'city'}, inplace=True)

In [ ]:
df_amazon_sr.head()

In [ ]:
df_amazon_sr['city'].nunique()

In [ ]:
# Check for hidden null values
df_amazon_sr[df_amazon_sr['city'].isin(['', ' '])]

In [ ]:
# Check for null values in 'city' and 'state' columns
df_amazon_sr[['city', 'state']].isnull().sum()

In [ ]:
# Investigate
df_amazon_sr[df_amazon_sr['state'].isnull()]

In [ ]:
# Drop row with 'Apo' city and missing state — cancelled and not monetized
df_amazon_sr = df_amazon_sr[~((df_amazon_sr['city'] == 'Apo') & (df_amazon_sr['state'].isnull()))]

In [ ]:
sorted(df_amazon_sr['city'].dropna().unique().tolist())

In [ ]:
# Check column order
df_amazon_sr.columns.tolist()

In [ ]:
# Reorder column names
df_amazon_sr = df_amazon_sr[
    [
        'date', 'order_id', 'shipping_method', 'status', 'fulfilment', 
        'courier_status', 'category', 'style', 'sku', 'size', 'qty',
        'amount', 'currency', 'city','state', 'postal_code',
        'latitude', 'longitude', 'promo_id', 'fulfilled_by', 'b2b'
    ]
]

In [ ]:
df_amazon_sr.head()

In [ ]:
# Check for duplicates
print(df_amazon_sr.duplicated().sum())

In [ ]:
# Inspect the 6 pairs of duplicates
dupes = df_amazon_sr[df_amazon_sr.duplicated(keep=False)]
display(dupes)

In [ ]:
# Check that they are identical
dupes.iloc[0].equals(dupes.iloc[1])

In [ ]:
# Drop them and check
df_amazon_sr.drop_duplicates(inplace=True)
print(df_amazon_sr.duplicated().sum())

In [ ]:
# Check missing values
missing = df_amazon_sr.isnull().sum()
missing = missing[missing > 0].sort_values(ascending=False)
display(missing)

In [ ]:
# Check missing lat/lon first
df_amazon_sr[df_amazon_sr['latitude'].isnull()][['postal_code', 'state', 'city']].head(10)

In [ ]:
missing_postals = df_amazon_sr[df_amazon_sr['latitude'].isnull()]['postal_code'].nunique()
print(f'Postal codes with missing lat/lon: {missing_postals}')

In [ ]:
# Note: Although 1,172 rows are missing lat/lon, they come from only 222 unique postal codes.
# Tableau aggregates by postal code, so these rows collapse into fewer map points.
# In Tableau, only 1 unmapped location appears — acceptable for now.
# Leaving these rows in the dataset in case we want to patch coordinates later.

In [ ]:
# Missing amount/currency
# Check to see if the 7665 rows with missing amount values were cancelled and that's why they are missing
df_amazon_sr[df_amazon_sr['amount'].isnull()]['status'].value_counts()

In [ ]:
# Most (7559) but not all (7665) were cancelled. 106 non-cancelled orders are missing currency/amount

In [ ]:
# Add a column to flag rows that have a valid (non-null) 'amount' value to track which orders contain real payment data
df_amazon_sr['has_amount'] = df_amazon_sr['amount'].notna()

In [ ]:
# Create a separate dataframe for completed, paid orders
df_amazon_sr_paid = df_amazon_sr[
    (df_amazon_sr['amount'].notna()) &
    (df_amazon_sr['status'] != 'Cancelled')
].copy() # prevent unwanted links to the original dataframe                             

In [ ]:
df_amazon_sr.head()

In [ ]:
# Fill missing courier_status values
# Check how many missing values exist in 'courier_status'
df_amazon_sr['courier_status'].isnull().sum()

In [ ]:
# Check to see if NaN values in courier_status are == status 'Cancelled'
df_amazon_sr[df_amazon_sr['courier_status'].isnull()]['status'].value_counts()

In [ ]:
# Of the 6869 missing courier_status values, 6858 are cancelled. Remaining 11 are in-between or post-ship states

In [ ]:
# Only fill NaNs in courier_status where status is "Cancelled", leave the 11 NaNs
df_amazon_sr.loc[
    (df_amazon_sr['courier_status'].isnull()) & (df_amazon_sr['status'] == 'Cancelled'),
    'courier_status'
] = 'Unshipped'

df_amazon_sr_paid.loc[
    (df_amazon_sr_paid['courier_status'].isnull()) & (df_amazon_sr_paid['status'] == 'Cancelled'),
    'courier_status'
] = 'Unshipped'

In [ ]:
# Check for suspicious rows: qty == 0 but status is not cancelled
df_amazon_sr[(df_amazon_sr['qty'] == 0) & (df_amazon_sr['status'] != 'Cancelled')].head()

In [ ]:
# Drop rows with qty == 0 that aren't cancelled (likely invalid)
df_amazon_sr = df_amazon_sr[~((df_amazon_sr['qty'] == 0) & (df_amazon_sr['status'] != 'Cancelled'))]

In [ ]:
# Clean up 'status' colums values

In [ ]:
df_amazon_sr['status'].unique()

In [ ]:
# Check all unique 'status' values and how many times each appears
df_amazon_sr['status'].value_counts(dropna=False)

In [ ]:
# Drop 6 rows with problematic shipping statuses — damaged, or lost in transit
problematic_statuses = [
    'Shipped - Damaged',
    'Shipped - Lost in Transit'
]

df_amazon_sr = df_amazon_sr[
    ~df_amazon_sr['status'].isin(problematic_statuses)
]
df_amazon_sr_paid = df_amazon_sr_paid[
    ~df_amazon_sr_paid['status'].isin(problematic_statuses)
]

In [ ]:
# Create a simplified 'status_clean' column for grouping and analysis

def clean_order_status(status, amount):
    if status.startswith('Cancelled'):
        return 'Cancelled'
    if status.startswith('Pending'):
        return 'Pending'
    if status in [
        'Shipped',
        'Shipped - Delivered to Buyer',
        'Shipped - Picked Up',
        'Shipped - Out for Delivery'
    ]:
        if amount == 0:
            return 'Shipped - Replacement'
        return 'Shipped'
    if status in [
        'Shipped - Returned to Seller',
        'Shipped - Rejected by Buyer',
        'Shipped - Returning to Seller'
    ]:
        return 'Returned'
    return 'Unknown'

In [ ]:
df_amazon_sr['status_clean'] = df_amazon_sr.apply(
    lambda row: clean_order_status(row['status'], row['amount']),
    axis=1
)

df_amazon_sr_paid['status_clean'] = df_amazon_sr_paid.apply(
    lambda row: clean_order_status(row['status'], row['amount']),
    axis=1
)

In [ ]:
# Cross-tab to check how status_clean aligns with courier_status
df_amazon_sr.groupby(['status_clean', 'courier_status']).size().unstack(fill_value=0)

In [ ]:
# Tag weird rows with new column status_mismatch
df_amazon_sr['status_mismatch'] = (
    ((df_amazon_sr['status_clean'] == 'Pending') & (df_amazon_sr['courier_status'] == 'Shipped'))
)
df_amazon_sr.head()

In [ ]:
df_amazon_sr_paid.groupby(['status_clean', 'courier_status']).size().unstack(fill_value=0)

In [ ]:
# Tag weird rows with new column status_mismatch
df_amazon_sr_paid['status_mismatch'] = (
        ((df_amazon_sr_paid['status_clean'] == 'Pending') & (df_amazon_sr_paid['courier_status'] == 'Shipped'))
)
df_amazon_sr_paid.head()

In [ ]:
# Check both dataframes: 
# df_amazon_sr should have null values only for currency/amount and lat/lon
# df_amazon_sr_paid should only have null for lat/lon

In [ ]:
df_amazon_sr.isnull().sum()

In [ ]:
df_amazon_sr_paid.isnull().sum()

In [ ]:
# Cleaning done! Reset the indexes. 
# Reset the index so it starts at 0 and removes the old index completely
df_amazon_sr.reset_index(drop=True, inplace=True)
# Reset the index so it starts at 0 and removes the old index completely
df_amazon_sr_paid.reset_index(drop=True, inplace=True)

In [ ]:
# Inspect the final clean versions!

In [ ]:
df_amazon_sr.describe()

In [ ]:
df_amazon_sr_paid.describe()

In [ ]:
df_amazon_sr.head()

In [ ]:
df_amazon_sr_paid.head()

In [ ]:
# We will use df_amazon_sr_paid for financial analysis (real, completed sales only)

In [ ]:
# For SKU-level or category-level trend analysis, sales volume, promo use, and time-based visualizations,
# we'll create a new version (df_amazon_sr_mean) with missing 'amount' values filled using the mean price per SKU

df_amazon_sr_mean = df_amazon_sr.copy()

# Flag rows where 'amount' was originally missing (for transparency in analysis)
df_amazon_sr_mean['amount_filled'] = df_amazon_sr_mean['amount'].isnull()

# Build a mapping of SKU -> average amount
sku_mean_map = (
    df_amazon_sr_mean.groupby('sku')['amount']
    .mean()
    .to_dict()
)

# Fill missing 'amount' values using the SKU-level mean
df_amazon_sr_mean['amount'] = df_amazon_sr_mean['amount'].fillna(
    df_amazon_sr_mean['sku'].map(sku_mean_map)
)

# Fill any remaining missing currency values with 'INR'
df_amazon_sr_mean['currency'] = df_amazon_sr_mean['currency'].fillna('INR')

In [ ]:
# Check null values, should be none (except lat/lon)
df_amazon_sr_mean.isnull().sum()

In [ ]:
# Inspect the 34 skus with null values for amount
df_amazon_sr_mean[df_amazon_sr_mean['amount'].isnull()]['sku'].value_counts()

In [ ]:
# Drop 34 remaining rows where 'amount' could not be calculated (no valid price history for the sku)
df_amazon_sr_mean = df_amazon_sr_mean[df_amazon_sr_mean['amount'].notna()]

In [ ]:
# Final check
df_amazon_sr_mean.isnull().sum()

In [ ]:
# Apply same function for the 'status' column to add clean 'order_status' column
df_amazon_sr_mean['status_clean'] = df_amazon_sr_mean.apply(
    lambda row: clean_order_status(row['status'], row['amount']),
    axis=1
)

In [ ]:
df_amazon_sr_paid.groupby(['status_clean', 'courier_status']).size().unstack(fill_value=0)

In [ ]:
# Tag weird rows with new column status_mismatch
df_amazon_sr_mean['status_mismatch'] = (
    ((df_amazon_sr_mean['status_clean'] == 'Pending') & (df_amazon_sr_mean['courier_status'] == 'Shipped'))
)
df_amazon_sr_mean.head()

In [ ]:
# Reset the index so it starts at 0 and removes the old index completely
df_amazon_sr_mean.reset_index(drop=True, inplace=True)

In [ ]:
# Confirm shape for each data frame
print ('df_amazon_sr',df_amazon_sr.shape)
print('df_amazon_sr_paid', df_amazon_sr_paid.shape)
print('df_amazon_sr_mean', df_amazon_sr_mean.shape)

In [ ]:
# # Save the clean dataframes as new .csv files

# df_amazon_sr.to_csv('cleaned_data/amazon_sales_clean.csv', index=False)
# df_amazon_sr_paid.to_csv('cleaned_data/amazon_sales_paid.csv', index=False)
# df_amazon_sr_mean.to_csv('cleaned_data/amazon_sales_mean.csv', index=False)

# # index=False keeps the row numbers out of the file for cleaner for loading later

In [ ]:
# This notebook cleaned the raw Amazon Sales Report CSV and produced three versions:
# 1. df_amazon_sr: full cleaned dataset (some included missing 'amount')
# 2. df_amazon_sr_paid: contains only valid, completed sales (non-cancelled + 'amount' present (use: financial analysis)
# 3. df_amazon_sr_mean: includes rows with filled 'amount'  using mean per sku (use: trend analysis)

In [ ]:
df_amazon_sr['order_id'].value_counts()

In [ ]:
df_amazon_sr['order_id'].nunique()